# X-ray spectroscopies

Tuning the photon energy to the X-ray regime, processes involving the transition of core electrons can be probed, providing measurements which are very sensitive to elements and local environment {cite}`xasxesbook2016, xrayrev2018, xfelbook2017`. Here we will discuss the modeling of four such spectroscopies, as illustrated in {numref}`xray-processes`:


```{figure} ../img/xray/processes.svg
---
name: xray-processes
---
Electronic processes involved in the X-ray sectroscopies discussed in this tutorial.
```

- In [X-ray photoemission spectroscopy (XPS)](sec:xray_calc_xps) the photoionization of the molecular system is probed, yielding insight into *occupied states*.

- In [X-ray absorption spectroscopy (XAS)](sec:xray_calc_xas) the transition of core electrons to occupied or continuum states is probed, providing information on *unoccupied states* or the *local chemical environment*.

- In [X-ray emission spectroscopy (XES)](sec:xray_calc_xes) the fluorescent decay of higher-lying electrons into a previously created core-hole is measured, yielding information on *occupied states*.

- In [resonant inelastic X-ray scattering (RIXS)](sec:xray_calc_rixs) (also termed resonant X-ray emission spectroscopy, or resonant X-ray Raman spectroscopy), the core-excitation and subsequent fluorescent decay is measured, yielding information on both *occupied and unoccupied states*, including the ability of probing *(local) vibrational modes*.

The element-specificity of these techniques arises from the significant differences in the energy of core orbital energies (and thus in excitation/fluorescence energies), as illustrated in {numref}`edges_elements`:

```{figure} ../img/xray/edges_elements.svg
---
name: edges_elements
---
Linear attenuation coefficient for the noble gases and for carbon to neon. Data extracted from [NIST](https://www.nist.gov/pml/x-ray-form-factor-attenuation-and-scattering-tables).
```

Here the linear attenuation coefficient for four noble gases (left), and carbon to neon (right) are shown, with the former including labels for the origin of each set of edge/set of features (using the Siegbahn notation with $n=1,2,3,...$ labeled $K, L, M, ...$). We see that there are significant energy differences between the various absorption features, which then enables us to focus on particular elements and edges by tuning the photon energy.

Modeling these processes is complicated by a number of factors, such as:

- Strong [relaxation effects](sec:xray_topics_relax), originating in the significant change in shielding of the nuclear charge arising from the creation/annihilation of a core-hole.

- Potentially significant impact from [relativity](sec:xray_topics_rel), with the strong potential experienced by the core electrons yielding large shifts due to (scalar) relativistic effects. In addition, spin-orbit coupling splits degeneracies in, *e.g.*, $2p$ (the $L_{2,3}$-edge), necessitating at least an approximate inclusion of such effects for these edges.

- Computational chemistry is more often concerned with valence properties, meaning that standard [basis sets](sec:xray_topics_basis) are not tailored for core-transition processes.

- For DFT method there are substantial influences due to the [self-interaction error](sec:xray_topics_sie), which are strong due to the compact nature of core orbitals.

- Core-excitation and -ionization processes are [embedded in a continumm of valence-ionized states](sec:xray_calc_cvs), making the application of, *e.g.*, a standard Davidson approach for resolving eigenstates unsuitable.

- The [localization/delocalization of core-holes](sec:xray_topics_chloc) for systems with symmetrically equivalent atoms can have a different impact on practical calculations, depending on the theory level.

- For higher photon energies the contribution of *multipole-allowed* states enables transitions which are electric-dipole forbidden.

Below we describe a typical workflow for calculating and analysing X-ray spectra, as well as an explicit example in terms of the IE, XAS, and XES of gaseous water. Following this, the tutorial will discuss in detail the [calculation](xray_calc.ipynb) of these X-ray spectra, focusing on how this is achieved using TDDFT and ADC. Following this, basic [analysis](xray_analysis.ipynb) schemes and the [aspects](xray_aspects.ipynb) raised above will be considered in some detail. The tutorial continues with a discussion on more exotic, [non-standard calculations](xray_nonstandard.ipynb), before closing with some general [recommendations](xray_rec.ipynb) for practical calculations, [exercises](xray_exercises.ipynb), and a [cheat sheet](xray_cheat) for the most common spectrum calculations.


## Typical workflow

- Perform [structure optimization](gsstructopt)
- Calculate the spectra/ionization energy
    - Chose suitable [level of theory](xray_rec)
    - Depending on spectroscopy:
        - [Ionization energy](sec:xray_calc_xps): using [$\Delta$-methods](sec:xray_calc_dscf), or target transitions to extremely [diffuse MOs](sec:xray_ie_diffuse)
        - [XAS](sec:xray_calc_xas): [CVS-ADC](sec:xray_calc_cvsadc), [CPP-DFT](sec:xray_calc_cppdft), or [CVS-TDSCF](sec:xray_calc_cvsdft)
        - [XES](sec:xray_calc_xes): [ADC](sec:xray_calc_xesadc), [TDSCF](sec:xray_calc_xesdft), and using [ground state MOs](sec:xray_calc_xesmos)
        - [RIXS](sec:xray_calc_rixs): two-step model with [ADC](sec:xray_calc_2stepadc)
- Analysis and assignment of the spectra
    - Looking at [amplitudes](sec:xray_anal_amp)
    - Visualization
        - [Molecular orbitals](visual-orbitals.ipynb)
        - [Natural transition orbitals](sec:vis_ntos)
        - [Attachment/detachment densities](sec:vis_attach_det)

## Example: gaseous water

Have the following X-ray absorption and emission specxtra of gaseous water, as well as ionization energy:

> Spectra and IE

Want to calculate these spectra and analyse the features.

### Structure optimization

Starting structure from Avogadro

Calculate with DFT (maybe xTB)

> Figure: overlaid structures from Avogadro and DFT optimized

Final structure:

```python
water_xyz = """
O       0.0000000000     0.0000000000     0.1178336003
H      -0.7595754146    -0.0000000000    -0.4713344012
H       0.7595754146     0.0000000000    -0.4713344012
"""
```

### Spectrum calculations

#### Level of theory

Go for ADC(2)-x, which works well for X-ray processes. IE from $\Delta$MP2.

cc-pVTZ is good. augment woth core-polarizing, and diffuse in order to get excited states

#### XAS

[CVS-ADC(2)-x](sec:xray_calc_cvsadc)

```python
# Construct structure and basis objects
struct = gator.get_molecule(water_xyz)
basis = gator.get_molecular_basis(struct, "6-31G")

# Perform SCF calculation
scf_gs = gator.run_scf(struct, basis)

# Calculate the 6 lowest eigenstates with CVS restriction to MO #1 (oxygen 1s)
adc_xas = gator.run_adc(
    struct, basis, scf_gs, method="cvs-adc2x", singlets=4, core_orbitals=1
)
```


#### XES

[ADC(2)-x](sec:xray_calc_xesadc)

```python
# Create pyscf mol object
mol = gto.Mole()
mol.atom = water_xyz
mol.basis = "6-31G"
mol.build()

# Perform unrestricted SCF calculation
scf_res = scf.UHF(mol)
scf_res.kernel()

# Copy molecular orbitals
mo0 = copy.deepcopy(scf_res.mo_coeff)
occ0 = copy.deepcopy(scf_res.mo_occ)

# Create 1s core-hole by setting alpha_0 population to zero
occ0[0][0] = 0.0

# Perform unrestricted SCF calculation with MOM constraint
scf_ion = scf.UHF(mol)
scf.addons.mom_occ(scf_ion, mo0, occ0)
scf_ion.kernel()

# Perform ADC calculation
adc_xes = adcc.adc2(scf_ion, n_states=4)
```

#### IE

Use the results we already have, using [$\Delta$-methods](sec:xray_calc_dscf)

```python
ie = adc_xas - adc_xes
```

#### Comparison to experiment

> Spectra (maybe tailor broadening, a bit)

### Spectrum analysis

#### IE

Not much is needed... Maybe MO

#### X-ray emission spectra

> print [amplitudes](sec:xray_anal_amp)

> dominated by one occupied each - MO analysis more or less enough

> All five [Molecular orbitals](visual-orbitals.ipynb)

#### X-ray absorption spectra

> print amplitudes

> MOs kinda works for first two features

> MOs - look above

For better: [Natural transition orbitals](sec:vis_ntos)

> NTOs of first three states (might skip the hole...)

Also: [Attachment/detachment densities](sec:vis_attach_det)

> attachmenet/detachment
